# Analysis of Germline Variants in Pancreatic Cancer

In this tutorial, we will analyze variants in Pancreatic cancer patients sequencing data of Earl,J et al. https://www.thelancet.com/journals/ebiom/article/PIIS2352-3964(20)30050-5/fulltext

    
![](images/paper.png)



### Overview of variant calling workflow:

![](images/workflow.png)


## Download the raw fastq files

We can download the raw fastq file on SRA website under Bioproject PRJNA644607 https://www.ncbi.nlm.nih.gov/sra?linkname=bioproject_sra_all&from_uid=644607

![](images/sra.png)

Download the raw fastq files using SRA-toolkit https://ncbi.github.io/sra-tools/fastq-dump.html

In [ ]:
./Downloads/sratoolkit.2.10.8-mac64/bin/fastq-dump -I --split-files SRR12165154

Let us look at the files that will be using

- `ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa` - the human reference genome (hg38)
- `SRR12165154_1.fastq` and `SRR12165154_2.fastq` - the query sequences

## Taking a peek at the reference fasta file

In [ ]:
head ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa

## Taking a look at the query sequence file (first 4 lines)

### One sequence in a fastq file consists of 4 lines. 

- Line 1 - sequence identifier (starts with @)
- Line 2 - DNA sequence
- Line 3 - sequence identifier (starts with +)
- Line 4 - corresponding quality score (Phred score 0-93 + 33)

For the quality score, the following characters encode the lowest to highest scores

<pre> !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ </pre>

For more information, see https://en.wikipedia.org/wiki/FASTQ_format


## Checking the quality of the FASTQ sequences

It is a good practice to check the quality of the sequences by plotting the quality (Q) scores by the position. In general, a Q score of > 30 is good.

To generate a plot, we will use `fastQC` (https://www.bioinformatics.babraham.ac.uk/projects/fastqc/)

In [ ]:
fastqc SRR12165154_1.fastq

In [ ]:
fastqc SRR12165154_2.fastq

it will generate an HTML file that we can open to see the quality scores.
The analysis modules can be seen on https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/


## Building the index for alignment


Before we can align the query sequence, we need to build the index for alignment. In this case, we will be using the `ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa` file

One of the programs for alignment is Bowtie2 (http://bowtie-bio.sourceforge.net/bowtie2/index.shtml)

Bowtie 2 is an ultrafast and memory-efficient tool for aligning sequencing reads to long reference sequences. It is particularly good at aligning reads of about 50 up to 100s or 1,000s of characters, and particularly good at aligning to relatively long (e.g. mammalian) genomes. Bowtie 2 indexes the genome with an FM Index to keep its memory footprint small: for the human genome, its memory footprint is typically around 3.2 GB. Bowtie 2 supports gapped, local, and paired-end alignment modes.


We will run `bowtie2` to see what the options are available

In [ ]:
bowtie2

Prior to the alignment, the reference genome must be indexed. 
This process may take several hours if indexing the full human genome (~4 GB).
The reference genome index already given in `ref` folder (6 files started with `hg38`)

In [ ]:
bowtie2-build ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa ref/hg38

## Aligning the query sequence

Now that we have built the reference index using Bowtie2, we can align the query sequences (`SRR12165154_1.fastq` and `SRR12165154_2.fastq`) to this reference.

In [ ]:
bowtie2 -x ref/hg38 -1 SRR12165154_1.fastq -2 SRR12165154_2.fastq -S SRR12165154.sam

## Looking at the SAM format

The SAM format is a tab delimited text file for storing alignments. The file usually starts with a header containing one/several lines marked by the letter `@`. This usually specifies the reference chromosomes used in the alignment, as well the the parameters used for the alignment

Following the header, each line of alignment consists of several tab-delimited columns.

<pre>QNAME FLAG RNAME POS MAPQ CIGAR MRNM MPOS ISIZE SEQ QUAL [TAG:VTYPE:VALUE[...]]</pre>
* The first 11 are mandatory
* Additional columns can be added using the format TAG:VTYPE:VALUE

From: http://zyxue.github.io/assets/sam_format_example.jpg
![](http://zyxue.github.io/assets/sam_format_example.jpg)

## Variant Calling

Now that we have the aligned sequences to the reference, we can look for difference between the aligned sequences and the reference. This process is also known as variant calling.

In order to perform variant calling, we need to preprocess the SAM file as well as the reference chromosome 5 fasta file. The following steps will be performed :

* Converting SAM to BAM format (this is the binary compressed version)
* Sorting the BAM file
* Marking duplicates in the BAM file
* Indexing the BAM file
* Indexing the reference fasta file

### Processing the SAM file generated by Bowtie2

We will process the SAM file using samtools, which can be accessed after loading the module.

In [ ]:
samtools view -Sb SRR12165154.sam -o SRR12165154.bam